In [1]:
import json
#from google.colab import files # solo para google collab
from google.cloud import storage
import pandas as pd
print(storage.__version__)

2.6.0


In [2]:
# google collab
#path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
path_api_key = '/Users/manuel/gdrive/prgrmmng/gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

In [3]:
# carga de datos
# unicamente para .py en VM:
#data_gmaps_reviews_norm = pd.read_parquet('gs://1_transform/data_gmaps_reviews_norm.parquet')

# para entorno Google Colab o maquina local:

# cambiar segun bucket
bucket = client_storage.get_bucket('1_transform')

# cambiar segun dataframe
data_blob = bucket.blob('data_gmaps_reviews_norm.parquet')
data_gmaps_reviews = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_gmaps_reviews.info()
data_gmaps_reviews.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115400 entries, 0 to 1115399
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   gmap_id          1115400 non-null  object 
 1   user_id          1115400 non-null  object 
 2   user_name        1115400 non-null  object 
 3   user_time        1115400 non-null  int64  
 4   rating           1115400 non-null  int64  
 5   user_text        685933 non-null   object 
 6   resp_time        181992 non-null   float64
 7   resp_text        181992 non-null   object 
 8   user_time_year   1115400 non-null  int32  
 9   user_time_month  1115400 non-null  int32  
 10  user_time_day    1115400 non-null  int32  
 11  user_time_hms    1115400 non-null  object 
 12  resp_time_year   181992 non-null   float64
 13  resp_time_month  181992 non-null   float64
 14  resp_time_day    181992 non-null   float64
 15  resp_time_hms    181992 non-null   object 
 16  state_name       1

,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,user_time_day,user_time_hms,resp_time_year,resp_time_month,resp_time_day,resp_time_hms,state_name,state_code,zcta5_geoid
0,0x88892bb449d44cb3:0x7beaf0b2e3171a49,110771718750038252970,Christian Buckner,1630031374725,2,"No masks, requested curbside and waited 10 min...",NaN,None,2021,8,27,02:29:34.725000,NaN,NaN,NaN,None,alabama,AL,35115
1,0x88892bb449d44cb3:0x7beaf0b2e3171a49,104877410053821600291,Susie Smith,1602297776261,5,This location does an awesome job! The delive...,NaN,None,2020,10,10,02:42:56.261000,NaN,NaN,NaN,None,alabama,AL,35115


In [64]:
columns_interes = [
    "state_name","state_code","zcta5_geoid",
    "gmap_id",
    "user_time_year","user_time_month","user_time_day",
    "rating"]

gmaps_kpi_popularidad = data_gmaps_reviews[
    columns_interes].sort_values(
        by=[
            "state_name","state_code","zcta5_geoid","gmap_id",
            "user_time_year","user_time_month","user_time_day"]
        ).copy() # type: ignore
gmaps_kpi_popularidad.head()

,state_name,state_code,zcta5_geoid,gmap_id,user_time_year,user_time_month,user_time_day,rating
279847,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1,3
278486,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,1,4
278472,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,2,3
278502,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,5,4
278466,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,6,4


In [65]:
gmaps_kpi_popularidad_rating_gmap_id = gmaps_kpi_popularidad.groupby(
    by=[
        "state_name","state_code","zcta5_geoid",
        "gmap_id","user_time_year","user_time_month"
        ],
    dropna=False
    )["rating"].size().reset_index().rename(columns={"rating":"gmap_id_rating_size"})
gmaps_kpi_popularidad_rating_gmap_id["gmap_id_size_pctchange"] = gmaps_kpi_popularidad_rating_gmap_id.groupby(
    by=[
        "state_name","state_code","zcta5_geoid",
        "gmap_id","user_time_year"
        ],
    dropna=False
    )["gmap_id_rating_size"].pct_change(1)*100
gmaps_kpi_popularidad_rating_gmap_id.head()

,state_name,state_code,zcta5_geoid,gmap_id,user_time_year,user_time_month,gmap_id_rating_size,gmap_id_size_pctchange
0,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1,NaN
1,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,8,NaN
2,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,8,5,-37.5
3,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,9,5,0.0
4,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,10,8,60.0


In [66]:
gmaps_kpi_popularidad_rating_zcta5_geoid = gmaps_kpi_popularidad.groupby(
     by=[
        "state_name","state_code","zcta5_geoid",
        "user_time_year","user_time_month"
        ],
    dropna=False
    )["rating"].size().reset_index().rename(columns={"rating":"zcta5_geoid_rating_size"})
gmaps_kpi_popularidad_rating_zcta5_geoid["zcta5_geoid_size_pctchange"] = gmaps_kpi_popularidad_rating_zcta5_geoid.groupby(
    by=[
        "state_name","state_code","zcta5_geoid",
        "user_time_year"
        ],
    dropna=False
    )["zcta5_geoid_rating_size"].pct_change(1)*100
gmaps_kpi_popularidad_rating_zcta5_geoid.head()

,state_name,state_code,zcta5_geoid,user_time_year,user_time_month,zcta5_geoid_rating_size,zcta5_geoid_size_pctchange
0,alabama,AL,30165,2020,7,56,NaN
1,alabama,AL,30165,2020,8,42,-25.000000
2,alabama,AL,30165,2020,9,66,57.142857
3,alabama,AL,30165,2020,10,63,-4.545455
4,alabama,AL,30165,2020,11,56,-11.111111


In [72]:
gmaps_kpi_popularidad_crecimiento = pd.merge(
    left=gmaps_kpi_popularidad_rating_gmap_id.drop(columns=["gmap_id_rating_size"]),
    right=gmaps_kpi_popularidad_rating_zcta5_geoid.drop(columns=["zcta5_geoid_rating_size"]),
    how='left'
).dropna()
gmaps_kpi_popularidad_crecimiento["kpi_popularidad_crecimiento"] = gmaps_kpi_popularidad_crecimiento[
    "gmap_id_size_pctchange"] - gmaps_kpi_popularidad_crecimiento["zcta5_geoid_size_pctchange"]
gmaps_kpi_popularidad_crecimiento.drop(
    columns=["gmap_id_size_pctchange","zcta5_geoid_size_pctchange"], inplace=True)
gmaps_kpi_popularidad_crecimiento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 237755 entries, 2 to 344450
Data columns (total 7 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state_name                   237755 non-null  object 
 1   state_code                   237755 non-null  object 
 2   zcta5_geoid                  237755 non-null  int64  
 3   gmap_id                      237755 non-null  object 
 4   user_time_year               237755 non-null  int32  
 5   user_time_month              237755 non-null  int32  
 6   kpi_popularidad_crecimiento  237755 non-null  float64
dtypes: float64(1), int32(2), int64(1), object(3)
memory usage: 12.7+ MB


In [71]:
gmaps_kpi_popularidad_parte = pd.merge(
    left=gmaps_kpi_popularidad_rating_gmap_id.drop(columns=["gmap_id_size_pctchange"]),
    right=gmaps_kpi_popularidad_rating_zcta5_geoid.drop(columns=["zcta5_geoid_size_pctchange"]),
    how='left'
).dropna()
gmaps_kpi_popularidad_parte["kpi_popularidad_parte"] = (
    gmaps_kpi_popularidad_parte["gmap_id_rating_size"] / gmaps_kpi_popularidad_parte["zcta5_geoid_rating_size"]
    )*100
gmaps_kpi_popularidad_parte.drop(
    columns=["gmap_id_rating_size","zcta5_geoid_rating_size"], inplace=True)
gmaps_kpi_popularidad_parte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344451 entries, 0 to 344450
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   state_name             344451 non-null  object 
 1   state_code             344451 non-null  object 
 2   zcta5_geoid            344451 non-null  int64  
 3   gmap_id                344451 non-null  object 
 4   user_time_year         344451 non-null  int32  
 5   user_time_month        344451 non-null  int32  
 6   kpi_popularidad_parte  344451 non-null  float64
dtypes: float64(1), int32(2), int64(1), object(3)
memory usage: 15.8+ MB


In [74]:
gmaps_kpi_popularidad = pd.merge(
    left=gmaps_kpi_popularidad_parte,
    right=gmaps_kpi_popularidad_crecimiento,
    how='left')
gmaps_kpi_popularidad.info()
gmaps_kpi_popularidad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344451 entries, 0 to 344450
Data columns (total 8 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state_name                   344451 non-null  object 
 1   state_code                   344451 non-null  object 
 2   zcta5_geoid                  344451 non-null  int64  
 3   gmap_id                      344451 non-null  object 
 4   user_time_year               344451 non-null  int32  
 5   user_time_month              344451 non-null  int32  
 6   kpi_popularidad_parte        344451 non-null  float64
 7   kpi_popularidad_crecimiento  237755 non-null  float64
dtypes: float64(2), int32(2), int64(1), object(3)
memory usage: 18.4+ MB


,state_name,state_code,zcta5_geoid,gmap_id,user_time_year,user_time_month,kpi_popularidad_parte,kpi_popularidad_crecimiento
0,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1.587302,NaN
1,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,14.285714,NaN
2,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,8,11.904762,-12.500000
3,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,9,7.575758,-57.142857
4,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,10,12.698413,64.545455
